In [1]:
import pandas as pd
import re

def extract_russian_name(text):
    """Извлекает русское имя с учетом всех исключений"""
    if not isinstance(text, str) or not text.strip():
        return None
    
    text = re.sub(r'^\s*Re:\s*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\(.*?\)', '', text)
    
    capital_words = re.findall(r'[А-ЯЁ][а-яё]+', text)
    if len(capital_words) > 1:
        match = re.search(
            r'(?:[А-ЯЁ][а-яё]+\s+){1,2}([А-ЯЁ][а-яё]+\s+[А-ЯЁ][а-яё]+)', 
            text
        )
        if match:
            return ' '.join(match.group(0).strip().split())
    return None

def clean_id_hash(text):
    """Удаляет статусные слова из id_hash"""
    if not isinstance(text, str):
        return text
        
    text = re.sub(
        r'\b(пропал[а-яё]*|жив[а-яё]*|погиб[а-яё]*|пропаж[а-яё]*)\b', 
        '', 
        text, 
        flags=re.IGNORECASE
    )
    return text.strip()

def process_file():
    try:
        # Чтение файла с определением кодировки
        try:
            df = pd.read_csv('updated_file.csv', encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv('updated_file.csv', encoding='cp1251')

        # Проверка обязательных колонок
        required_columns = [
            'id_hash', 'gender', 'age', 'location', 'status',
            'date_search', 'date_of_loss', 'last_search_date',
            'search_period', 'head', 'date_of_find'
        ]
        
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Ошибка: отсутствуют колонки {missing_columns}")
            return

        # Сохраняем оригинальные данные для статистики
        initial_count = len(df)
        
        # Обработка имен
        df['original_id_hash'] = df['id_hash']
        df['extracted_name'] = df['head'].apply(extract_russian_name)
        df.loc[df['extracted_name'].notna(), 'id_hash'] = df['extracted_name']
        
        # Очистка от статусных слов
        df['id_hash'] = df['id_hash'].apply(clean_id_hash)

        # Поля для определения дубликатов
        duplicate_columns = [
            'id_hash', 'gender', 'age', 'location', 'status',
            'date_search', 'date_of_loss', 'last_search_date',
            'search_period', 'head', 'date_of_find'
        ]
        
        # Удаление дубликатов
        duplicates_mask = df.duplicated(subset=duplicate_columns, keep='first')
        removed_count = duplicates_mask.sum()
        df = df[~duplicates_mask]

        # Удаление служебных колонок
        df.drop(
            columns=['original_id_hash', 'extracted_name'],
            inplace=True,
            errors='ignore'
        )

        # Сохранение результата
        df.to_csv('updated_file.csv', index=False, encoding='utf-8')
        
        print("Файл успешно обработан. Статистика:")
        print(f"Всего строк до обработки: {initial_count}")
        print(f"Удалено дубликатов: {removed_count}")
        print(f"Оставлено уникальных записей: {len(df)}")
        print("\nПример обработанных данных:")
        print(df.head())

    except Exception as e:
        print(f"Ошибка обработки: {str(e)}")

if __name__ == "__main__":
    process_file()

Файл успешно обработан. Статистика:
Всего строк до обработки: 40448
Удалено дубликатов: 431
Оставлено уникальных записей: 40017

Пример обработанных данных:
   Unnamed: 0                                            id_hash gender   age  \
0           0  002a7a0fd2047c4812ee06974cfec87266c73307ac4d29...    жен  65.0   
1           1  008c48640c96927ecb37bf877179e2e506052ea235e4d7...    муж  38.0   
2           2  00fdf06a413b627f8a53c77ce9dcde274b94aa895d6f15...    муж   NaN   
3           3  012742c048aa46328fbca7f18376b6db990499a31ca158...    муж  69.0   
4           4  0228b07e42be793dc3259b75fe30468085922a0959c24f...    жен  45.0   

          location     status         date_search  date_of_loss  \
0              NaN  пропал(а)  19 сен 2023, 00:42           NaN   
1  пгт. Кавалерово  пропал(а)  12 июн 2024, 06:25   9 июня 2024   
2              NaN        NaN  15 авг 2018, 02:00           NaN   
3              NaN  пропал(а)  15 сен 2023, 06:57  24 июня 2023   
4              NaN   